<img src="https://toppng.com/uploads/preview/linkedin-logo-png-photo-116602552293wtc4qogql.png" width="20" height="20" /> [Bharath Hemachandran](https://www.linkedin.com/in/bharath-hemachandran/)

# 🧭 Phase 4: Agent Workflows & Multi-Agent Design

In Phase 3 you built a **single agent** that could use tools (via MCP). In this phase, you’ll learn how to design **agent workflows** and build a simple **multi-agent workflow** in code.

<div style="background: #e8f5e9; padding: 14px; border-radius: 8px; border-left: 4px solid #2e7d32;">
<strong>🎯 What you'll do:</strong> Understand common agent workflow patterns, learn how to pick the right one for your use case and tech stack, design personas and human-in-the-loop checkpoints, and finally implement a small multi-agent workflow using Groq + MCP.
</div>

### 📋 Notebook objective (table of contents)

This notebook covers:
- **Recap & segue from Phase 3** — From single agent to workflows
- **Agent workflow types** — Sequential, parallel, and coordinated patterns in detail
- **How to choose a workflow type** — Questions to ask; concrete examples where one type beats another
- **Exercise** — Choose the best workflow type for given scenarios (with suggested answers)
- **Designing a workflow** — Breaking work into phases and personas (step-by-step)
- **Framework choices** — How LangChain, LlamaIndex, crewAI, AutoGen differ
- **Human-in-the-loop (HITL)** — Where to place humans; design choices (granularity, blocking vs async)
- **Code lab** — Implement a simple multi-agent workflow with Groq + MCP
- **Exercises (end of phase)** — Sequential vs parallel, HITL placement, two-persona design
- **Phase 4 wrap-up** — Summary and next steps


## 🔁 From a single agent (Phase 3) to workflows

In **Phase 3**, you:
- Wrapped Groq’s `responses.create` + an MCP tool into a **single-agent helper**.
- Let the model **decide when to call the Hugging Face MCP tool**.
- Inspected `mcp_call` items to see how the agent used tools.

That’s already a minimal **agent**: one loop around a model that can decide to use tools.

In this phase we zoom out and design a **workflow**:
- Multiple **steps/phases** (e.g. research → evaluation → decision).
- Optionally multiple **personas/agents**, each with their own role and tools.
- **Human-in-the-loop** checkpoints for safety and control.


## 🧱 Agent workflow types (patterns)

Workflows differ in **how agents are ordered** and **how they share information**. Three high-level categories:

---

### 1. **Sequential workflows**

Agents run **one after another** in a fixed order. Output of step *N* becomes input to step *N+1*.

- **Flow:** `Agent A → Agent B → Agent C → …`
- **Data:** Each step consumes the previous step’s output (and optionally the original user goal).
- **Use when:** The task has a natural pipeline (e.g. research → evaluate → recommend), dependencies are clear, and you want predictable, auditable steps.
- **Trade-offs:** Easy to reason about and debug; latency adds up (no parallelism). Best with a **single strong model** per step or **specialist models** (e.g. one for search, one for summarization).

**Examples:** Document ingestion (parse → extract → classify), model selection (research → evaluate → recommend), content pipelines (research → draft → review).

---

### 2. **Parallel workflows**

Multiple agents (or branches) run **at the same time**, on the same or different inputs. Results are merged later.

- **Flow:** Fork into *N* branches; each branch runs independently; then join (e.g. vote, rank, or concatenate).
- **Data:** Same input broadcast to all branches, or different slices (e.g. by topic). Merge step combines outputs.
- **Use when:** You need **diversity** (e.g. multiple draft answers to pick from), **coverage** (e.g. analyze 5 documents in parallel), or **redundancy** (e.g. multiple reviewers). Best when steps are **independent** and you can afford multiple model calls.
- **Trade-offs:** Lower end-to-end latency than doing the same steps sequentially; higher cost per request. Merge logic (voting, ranking, summarization) must be designed carefully.

**Examples:** Multi-perspective summarization (each agent summarizes one section in parallel, then merge), parallel code reviews (several agents review the same PR), A/B-style answer generation (generate 3 answers in parallel, then rank).

---

### 3. **Coordinated workflows**

A **coordinator** (or router) decides **which agent(s)** to call, in **what order**, and possibly **multiple rounds**. Flow is dynamic, not fixed.

- **Flow:** Coordinator receives goal → chooses next agent(s) and task(s) → agents run (possibly in parallel) → coordinator gets results → repeats or finishes.
- **Data:** Coordinator maintains state (e.g. plan, partial results, history); agents receive sub-tasks and return structured or free-form results.
- **Use when:** The task is **open-ended** or **multi-step with branching** (e.g. “answer this question using whatever tools you need”), or when you want **one agent to delegate** to specialists. Fits **planner–executor** and **tool-using single agent** patterns.
- **Trade-offs:** Flexible and powerful; harder to debug and more variable in cost/latency. Often implemented with a **single model that has access to tools** (including “call another agent” as a tool) or a **dedicated coordinator model** that outputs a plan.

**Examples:** Research assistant (plan → search → read → synthesize → maybe search again), helpdesk (classify → route to specialist agent → maybe escalate), iterative refinement (draft → critique → revise until satisfied).

---

### Summary table

| Pattern        | Order of execution | Info flow           | Best for                                      |
|----------------|--------------------|---------------------|-----------------------------------------------|
| **Sequential** | Fixed, step-by-step| A → B → C           | Pipelines, audits, clear dependencies         |
| **Parallel**   | Multiple at once   | Fork → merge        | Diversity, coverage, independent sub-tasks   |
| **Coordinated**| Dynamic, by coordinator | Coordinator ↔ agents | Open-ended tasks, planning, delegation |

---

### Other patterns (brief)

- **Single-call tool-using agent** — One model call can use tools (like Phase 3). Good for small, well-scoped tasks; no explicit “workflow,” just one intelligent step.
- **Planner–executor loop** — A form of coordinated workflow: one agent outputs a plan; another (or the same) executes steps, possibly with tools, in a loop.
- **Agent in a larger app** — Any of the above embedded in APIs, queues, or schedulers; adds observability, retries, and SLAs.


## 🎯 How to choose a workflow type

Ask these questions before choosing a pattern:

- **Task complexity** — One-shot question vs. multi-step project with dependencies.
- **Determinism vs exploration** — Predictable path (pipelines) vs. flexible exploration (coordinated / planner).
- **Latency & cost** — How many model calls per request you can afford.
- **Risk & safety** — Whether you need human approval before certain actions (e.g. sending emails, committing code).
- **Data & tools** — Which MCP servers, APIs, or DBs must be used; local vs. remote.
- **Team / org mapping** — Whether you can mirror existing human roles (researcher, reviewer, approver) as agent personas.

---

### When one type is better than another: concrete examples

Below are **concrete use cases** and why a given **workflow type** (and sometimes **model choice**) fits better.

| Use case | Better fit | Why the other options are worse |
|----------|------------|----------------------------------|
| **“Find a Hugging Face model for task X and recommend one”** | **Sequential** (research → evaluate → recommend) | Dependencies are clear: you must research before evaluating. Parallel doesn’t help (evaluation needs research output). Coordinated is overkill for this linear process. |
| **“Summarize these 10 long documents”** | **Parallel** (one agent per doc, then merge) | Each doc is independent; parallelizing cuts latency. Sequential would be slow (10 steps in a row). Coordinated adds complexity without benefit. |
| **“Answer this question using the web and our internal docs; use whatever steps you need”** | **Coordinated** (single agent with tools, or planner–executor) | Steps are unknown in advance (search → maybe read → maybe search again). Sequential assumes a fixed pipeline; parallel assumes independent branches. A tool-using or planner agent fits best. |
| **“Generate 5 tagline options for our product; pick the best one”** | **Parallel** (5 agents in parallel, then one ranker) | You want diversity (5 different options) then a single decision. Sequential would give one option per step and no direct comparison. Coordinated could do it but parallel is simpler and faster. |
| **“Review this PR: security, style, and performance”** | **Parallel** (3 specialist reviewers in parallel, then merge) | Three independent review dimensions; running in parallel is faster and keeps roles clear. Sequential would delay the later reviews; coordinated adds orchestration you don’t need. |
| **“Draft a report, then have a critic improve it, then have an editor polish it”** | **Sequential** (draft → critique → edit) | Each step depends on the previous. Parallel can’t work (critic needs draft; editor needs revised draft). Coordinated could emulate this but sequential is the natural fit. |
| **“Handle customer ticket: classify, then route to the right specialist, maybe escalate”** | **Coordinated** (classifier → specialist agents; coordinator can re-route or escalate) | Routing and escalation are dynamic. Sequential would force a fixed path; parallel doesn’t model routing. A coordinator that chooses the next agent fits best. |
| **“One-shot: what’s the weather in Paris?”** | **Single-call tool-using agent** (one model + one tool call) | No multi-step workflow; one call with a tool is enough. Sequential/parallel/coordinated add unnecessary complexity and cost. |

---

### Model architecture and workflow

- **Sequential:** Often **one model per step** (same or different). You can use a smaller/faster model for simple steps (e.g. classification) and a larger one for complex steps (e.g. summarization).
- **Parallel:** Same model replicated across branches is common; or **specialist models** per branch (e.g. security reviewer vs. style reviewer). Throughput and cost scale with the number of branches.
- **Coordinated:** Often **one powerful model** with tools (including “call another agent” or “run sub-task”); or a **lightweight coordinator** that calls specialist models. The coordinator’s prompt and tool set define the possible behaviors.

For this notebook, we’ll implement a **sequential multi-agent pipeline** (research → evaluate → recommend) with clear personas and explicit human checkpoints.


## ✏️ Exercise: Choose the workflow type

For each scenario below, decide which workflow type fits best: **sequential**, **parallel**, or **coordinated** (or **single-call** if it’s a one-shot task). Justify in one or two sentences. Optionally, sketch the steps (e.g. “Agent A → Agent B → merge”).

1. **Scenario A:** “We have 50 support tickets. For each ticket, classify intent (billing / technical / feedback), then generate a first draft reply. We want results as fast as possible.”
2. **Scenario B:** “We want one final blog post. First research 3 competitor articles, then draft an outline, then write the post, then run a fact-check pass.”
3. **Scenario C:** “User asks a free-form question. We want to use web search and our knowledge base, and we don’t know in advance how many searches or reads we’ll need.”
4. **Scenario D:** “We need 3 alternative ad copy options for a campaign. We’ll then pick one manually. No dependency between the three options.”
5. **Scenario E:** “Process incoming invoices: extract line items (OCR + parser), validate against PO database, then flag exceptions for human review.”

**Suggested answers (try the exercise first, then peek):**

- **A:** **Parallel** — 50 independent tickets; classify + draft per ticket in parallel (or parallel over tickets) to minimize latency. Merge could be “collect all results” or a final QA pass.
- **B:** **Sequential** — research → outline → write → fact-check. Each step depends on the previous; fixed pipeline.
- **C:** **Coordinated** — open-ended; number and order of tool calls (search, read) are unknown. Single agent with tools or planner–executor fits.
- **D:** **Parallel** — 3 independent draft agents in parallel; human picks one. No need for sequential or coordinator.
- **E:** **Sequential** — extract → validate → flag. Clear pipeline with dependencies; human-in-the-loop at the end for exceptions.

## 🧑‍🎓 Designing personas & phases

Breaking a workflow into **phases** and **personas** makes behavior predictable and easier to debug. Here’s how to do it in more detail.

---

### What is a persona?

A **persona** is a specialized agent with:

- **Role** — What it is responsible for (e.g. *Model Researcher*, *Evaluator*, *Writer*).
- **Goals** — How success is defined (e.g. *find 3–5 candidate models*, *score each from 1–10*).
- **Tools** — Which MCP servers, APIs, or data sources it can use (e.g. Hugging Face MCP, web search, internal DB).
- **Constraints** — Style, safety rules, output format, and limits (e.g. “output markdown only,” “do not call external APIs without approval”).

Personas map well to **existing human roles** (researcher, reviewer, approver), which helps with handoffs and human-in-the-loop.

---

### Step-by-step design process

1. **Define the end-to-end goal.**  
   Be specific: “Pick a Hugging Face text-generation model for marketing copy with low latency and a permissive license” is better than “find a good model.”

2. **List the logical steps (phases).**  
   - What must happen first? (e.g. gather options.)  
   - What depends on that? (e.g. evaluate options against criteria.)  
   - What comes last? (e.g. recommend and explain.)

3. **Assign a persona to each phase.**  
   - Give each persona a **name** (e.g. `ResearcherAgent`, `EvaluatorAgent`) and a **one-line responsibility**.  
   - Decide **which tools** each persona can use. Not every agent needs tools (e.g. EvaluatorAgent might only reason over text).

4. **Define inputs and outputs per step.**  
   - ResearcherAgent: input = user goal; output = structured list of candidates.  
   - EvaluatorAgent: input = user goal + research summary; output = scores + final recommendation.  
   This clarifies handoffs and makes it easy to add human checkpoints (e.g. “human approves research summary before it goes to EvaluatorAgent”).

5. **Decide where humans must be involved.**  
   - After research: human can refine scope or add constraints.  
   - Before final recommendation: human can approve, reject, or request another iteration.  
   - Optionally: human approves each tool call (e.g. `require_approval` for sensitive MCP tools).

---

### Example: our model-selection workflow

| Phase   | Persona           | Input              | Output                    | Tools        |
|---------|--------------------|--------------------|---------------------------|--------------|
| 1       | ResearcherAgent    | User goal          | 3–5 candidates (summary)  | Hugging Face MCP |
| (HITL)  | Human              | Research summary   | Approve / refine / stop   | —            |
| 2       | EvaluatorAgent     | Goal + research    | Scores + recommendation   | None         |
| (HITL)  | Human              | Evaluation         | Approve / stop            | —            |
| 3       | Coordinator        | Approved output    | Final recommendation      | —            |

We implement “Coordinator” as the Python function that runs the pipeline and calls the human at each checkpoint; it doesn’t need a separate model call.


## 🧰 Framework choices: how they differ

You can implement these workflows in different ways:

| Framework / style | Strengths | When to use |
|-------------------|-----------|-------------|
| **Groq + MCP (DIY)** | Direct control over prompts, tools, and loops; minimal dependencies. | Teaching, prototypes, or when you want to understand the mechanics (like this notebook). |
| **LangChain** | Rich library of tools, agents, memory, and chains; many integrations. | When you want batteries-included tooling and flexible orchestration in Python/JS. |
| **LlamaIndex** | Strong focus on indexes, retrieval, and data-centric agents. | When your workflow centers on documents, RAG, and structured data sources. |
| **crewAI** | Multi-agent “crew” with roles, goals, and tasks; built-in coordination patterns. | When you want opinionated multi-agent patterns with less manual orchestration. |
| **AutoGen** | Conversational multi-agent framework (agents talk to each other). | When you want agents to negotiate, critique, or co-create via dialogue. |

In this notebook we use **Groq + MCP directly** so you can see the underlying pattern that other frameworks abstract over.


## 🙋 Human-in-the-loop (HITL): where to place humans

Humans can appear at several points in an agent workflow. Choosing the **right places** balances autonomy (fewer interruptions) with safety and control (more oversight where it matters).

---

### Where to add HITL checkpoints

| Checkpoint type      | When to use it | What the human does |
|----------------------|----------------|----------------------|
| **Task definition**  | Start of workflow | Provide or edit the initial goal, constraints, or scope. |
| **Tool approval**    | Before sensitive tool calls | Approve or reject each call to tools that change state (e.g. send email, write to DB, call paid APIs). With MCP, this is often modeled via `require_approval` or connector settings. |
| **Intermediate review** | After a phase (e.g. research, draft) | Inspect output; approve to continue, refine the prompt, or stop. Prevents bad intermediate results from flowing to the next agent. |
| **Final approval**   | End of workflow | Approve the final recommendation or output before it is applied (e.g. before publishing, deploying, or sending to a customer). |
| **Escalation**       | When the agent is uncertain | Agent explicitly hands control back (e.g. “I couldn’t find a match; please clarify”). Human resolves and may re-enter the workflow. |

---

### Design choices

- **Granularity:** Approval per tool call (fine-grained, more friction) vs. approval per phase (coarser, fewer interruptions). Use per-phase for most workflows; use per-tool for high-risk actions.
- **Blocking vs. async:** Blocking = workflow pauses until the human responds (e.g. console `input()`). Async = workflow continues and notifies a human (e.g. ticket, Slack); human can intervene later. Our code example uses **blocking** console prompts for simplicity.
- **Default behavior:** If the human doesn’t respond in time, decide: auto-continue, auto-stop, or retry. Document this in your runbooks.

In our code example we implement **two intermediate checkpoints** (after research, after evaluation) using simple **blocking console prompts** (y/n). You can replace these with UI buttons, tickets, or approval workflows in production.


## 🔧 Setup (run once)

Same as previous phases: install **openai**, set **GROQ_API_KEY** (and optional **HF_TOKEN**), and create a Groq client. Then we’ll reuse the Hugging Face MCP tool from Phase 2–3.


In [ ]:
!pip install -q openai


In [ ]:
import os
from getpass import getpass

if not os.environ.get("GROQ_API_KEY"):
    os.environ["GROQ_API_KEY"] = getpass("Paste your GROQ_API_KEY: ")
if not os.environ.get("HF_TOKEN"):
    tok = getpass("Paste your HF_TOKEN (or press Enter to skip): ")
    if tok:
        os.environ["HF_TOKEN"] = tok

from openai import OpenAI

def get_groq_client():
    return OpenAI(
        api_key=os.environ["GROQ_API_KEY"],
        base_url="https://api.groq.com/openai/v1",
    )

HF_TOKEN = os.environ.get("HF_TOKEN")
print("✅ Groq client ready." if os.environ.get("GROQ_API_KEY") else "Need GROQ_API_KEY.")


## 🔧 MCP tool (Hugging Face) — same as Phase 2–3

We reuse the same Hugging Face MCP tool from Phase 2–3. Only the **ResearcherAgent** will use it; the **EvaluatorAgent** will reason over text only (no tools).

In [ ]:
mcp_tool = {
    "type": "mcp",
    "server_label": "Huggingface",
    "server_url": "https://huggingface.co/mcp",
    "server_description": "Search and access AI models from Hugging Face",
    "require_approval": "never",  # for this lab we automate tool calls
}
if HF_TOKEN:
    mcp_tool["headers"] = {"Authorization": f"Bearer {HF_TOKEN}"}

print("MCP tool: Hugging Face")


## 🧩 Designing our example multi-agent workflow

We’ll implement a **3-persona workflow** around Hugging Face model selection:

- **User goal** (input): e.g. “I need a text-generation model for marketing copy with low latency and a permissive license.”
- **Agent 1 – ResearcherAgent**: uses Hugging Face MCP to find 3–5 candidate models and summarize them.
- **Human checkpoint #1**: you review/refine the research summary.
- **Agent 2 – EvaluatorAgent**: takes the refined research + requirements and scores/ranks candidates.
- **Human checkpoint #2**: you approve the evaluation or ask for another pass.
- **Coordinator**: packages a final recommendation.

We’ll keep all of this in one notebook and implement agents as **small helper functions** around `responses.create`.


In [ ]:
client = get_groq_client()

def run_agent(role_name: str, role_instructions: str, task: str, tools=None):
    """Generic helper to run an agent with a role, instructions, and a task.

    - role_name: label used in logs (e.g. "ResearcherAgent").
    - role_instructions: how this agent should behave.
    - task: natural-language task description.
    - tools: list of tools this agent can use (e.g. [mcp_tool]).
    """
    full_prompt = (
        f"You are {role_name}.\n"  # light system-style prefix
        f"{role_instructions}\n\n"
        f"Task: {task}"
    )
    response = client.responses.create(
        model="openai/gpt-oss-120b",
        input=full_prompt,
        tools=tools or [],
    )
    return response

print("✅ Agent helper ready.")


## 🧪 Multi-agent workflow with human-in-the-loop

The function below implements our **sequential** pipeline: ResearcherAgent (with MCP) → human checkpoint → EvaluatorAgent (no tools) → human checkpoint → final recommendation. Set `hitl=False` to run without console prompts (e.g. for automated demos).

In [ ]:
def run_multi_agent_workflow(user_goal: str, hitl: bool = True):
    """Run a simple Researcher → Evaluator pipeline with optional human approval.

    Returns the final recommendation text.
    """
    print("=== USER GOAL ===")
    print(user_goal)
    print("\n=== STEP 1: ResearcherAgent (uses Hugging Face MCP) ===")

    researcher_instructions = (
        "You are a model researcher using the Hugging Face Hub. "
        "Use the available tools to find 3–5 candidate models that could meet the user's goal. "
        "For each candidate, capture: model id, task, license, rough size, and why it might fit. "
        "Return your findings as a concise markdown bullet list."
    )

    researcher_resp = run_agent(
        role_name="ResearcherAgent",
        role_instructions=researcher_instructions,
        task=user_goal,
        tools=[mcp_tool],
    )

    researcher_summary = researcher_resp.output_text
    print("\n--- ResearcherAgent output ---\n")
    print(researcher_summary)

    if hitl:
        cont = input("\nApprove research summary? [y to continue, anything else to stop]: ")
        if cont.strip().lower() != "y":
            print("\n⏹ Workflow stopped by human after research phase.")
            return researcher_summary

    print("\n=== STEP 2: EvaluatorAgent (scores candidates vs constraints) ===")

    evaluator_instructions = (
        "You are an evaluation expert. Given the user's goal and the researcher's "
        "candidate list, score each candidate from 1–10 for fit. "
        "Consider task match, license, community adoption, and likely latency. "
        "Return: (1) a short table of candidates with scores, (2) a clear final recommendation."
    )

    evaluator_task = (
        "User goal: " + user_goal + "\n\n" +
        "Research summary from ResearcherAgent:\n" + researcher_summary
    )

    evaluator_resp = run_agent(
        role_name="EvaluatorAgent",
        role_instructions=evaluator_instructions,
        task=evaluator_task,
        tools=[],  # this agent just reasons over the researcher's text
    )

    evaluator_summary = evaluator_resp.output_text
    print("\n--- EvaluatorAgent output ---\n")
    print(evaluator_summary)

    if hitl:
        cont2 = input("\nApprove evaluation and final recommendation? [y to accept, anything else to stop]: ")
        if cont2.strip().lower() != "y":
            print("\n⏹ Workflow stopped by human after evaluation phase.")
            return evaluator_summary

    print("\n✅ Workflow complete. Returning final recommendation.")
    return evaluator_summary

print("✅ Multi-agent workflow function ready.")


In [ ]:
user_goal = (
    "I need a Hugging Face text-generation model to generate short English marketing "
    "taglines for a consumer app. Latency should be low (fast inference), and the license "
    "should allow commercial use."
)

final_recommendation = run_multi_agent_workflow(user_goal, hitl=True)

print("\n=== FINAL RECOMMENDATION (from workflow) ===\n")
print(final_recommendation)


## 🚀 Run the multi-agent workflow

Edit `user_goal` in the cell above and rerun to try different model-selection requests. With `hitl=True`, you’ll be prompted to approve after the research phase and after the evaluation phase.

## ✏️ Exercises (end of phase)

*Use only what you learned in this phase (workflow types, choosing workflows, personas, HITL, multi-agent code).*

1. **Sequential vs parallel**  
   For the task *"Generate 3 tagline options for our product and then pick the best one,"* would you choose a **sequential** or **parallel** workflow? Justify in one or two sentences (think about dependencies and diversity).

2. **Where to add a human checkpoint**  
   In our Researcher → Evaluator workflow, the final output is a model recommendation. If that recommendation might be **shown directly to a customer**, where would you add a human-in-the-loop checkpoint (e.g. after research, after evaluation, or both)? Give a short reason.

3. **Design two personas**  
   Sketch a **2-step persona design** for: *"Answer a support ticket: first classify the intent (billing / technical / feedback), then draft a short reply."* For each step give: (a) a **role name**, (b) **input** and **output**, and (c) whether that step needs a **tool** (e.g. MCP) or only the model. You can write it in a small table or bullet list.

## ✅ Phase 4 wrap-up

In this phase you:
- Learned common **agent workflow patterns** and how to choose among them.
- Designed **personas and phases** for a model-selection workflow.
- Discussed how different **frameworks** (LangChain, LlamaIndex, crewAI, AutoGen) map to these ideas.
- Implemented a simple **multi-agent pipeline with human-in-the-loop** using Groq + MCP.

You can now adapt this pattern to your own use cases: change the goal, personas, tools, or human checkpoints, or port the same design into a higher-level agent framework.
